<a href="https://colab.research.google.com/github/ribalchusy/machine-learning/blob/main/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №6: Машины опорных векторов

In [1]:
import numpy as np
# фреймы данных
import pandas as pd
# графики
import matplotlib as mpl
# стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# перекодировка символьных показателей
from sklearn.preprocessing import LabelEncoder
# для таймера
import time
# загрузка пакетов: модели -------------------------------------------------
# SVM
from sklearn.svm import SVC
# логистическая рагрессия
from sklearn.linear_model import LogisticRegression
# стандартизация
from sklearn.preprocessing import StandardScaler
# метод главных компонент
from sklearn.decomposition import PCA
# конвейеры
from sklearn.pipeline import make_pipeline
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# сводка по точности классификации
from sklearn.metrics import classification_report

In [2]:
# константы
# ядро для генератора случайных чисел
my_seed = 15
# создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
# примеры стилей и шаблонов графиков:
# http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
# sns.color_palette("Set2")

<ipython-input-2-740a19711f8b>:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mpl.style.use('seaborn-whitegrid')


# Загружаем данные
Загружаем данные во фрейм и выясняем их размерность. Смотрим первые строки таблицы

In [4]:
# загружаем таблицу и превращаем её во фрейм
fileURL = 'default_of_credit_card_clients.csv'
DF_all = pd.read_csv(fileURL)
# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_all.shape)

Число строк и столбцов в наборе данных:
 (30000, 24)


In [5]:
# оставляем в наборе данных только то, что нужно
DF_all = DF_all[['Y' , 'LIMIT_BAL' , 'SEX' , 'EDUCATION' , 'MARRIAGE' , 'AGE' , 'PAY_0' , 'PAY_2' , 'PAY_3' ,'BILL_AMT1' , 'BILL_AMT2' , 'BILL_AMT3' , 'PAY_AMT1' ,  'PAY_AMT2'  , 'PAY_AMT3']]
DF_all.head(5)

,Y,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,BILL_AMT1,BILL_AMT2,BILL_AMT3,PAY_AMT1,PAY_AMT2,PAY_AMT3
0,1,20000,2,2,1,24,2,2,-1,3913,3102,689,0,689,0
1,1,120000,2,2,2,26,-1,2,0,2682,1725,2682,0,1000,1000
2,0,90000,2,2,2,34,0,0,0,29239,14027,13559,1518,1500,1000
3,0,50000,2,2,1,37,0,0,0,46990,48233,49291,2000,2019,1200
4,0,50000,1,2,1,57,-1,0,-1,8617,5670,35835,2000,36681,10000


отложим 15% данных для моделирования

In [6]:
# наблюдения для моделирования
DF = DF_all.sample(frac = 0.85, random_state = my_seed)
# отложенные наблюдения
DF_predict = DF_all.drop(DF.index)

# Преобразование исходных данных и построение моделей


In [8]:
DF.describe()


,Y,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,BILL_AMT1,BILL_AMT2,BILL_AMT3,PAY_AMT1,PAY_AMT2,PAY_AMT3
count,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,2.550000e+04,25500.000000
mean,0.221137,167607.137255,1.602588,1.854627,1.549294,35.524471,-0.014510,-0.133176,-0.165922,51195.828314,49147.973490,46796.098549,5621.995294,5.781955e+03,5214.019020
std,0.415021,129517.884174,0.489372,0.790401,0.522187,9.234218,1.126088,1.200379,1.197422,73529.824352,70945.246561,68468.522521,16414.783302,2.027515e+04,17129.286318
min,0.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-157264.000000,0.000000,0.000000e+00,0.000000
25%,0.000000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,3476.500000,2975.000000,2643.000000,1000.000000,8.240000e+02,390.000000
50%,0.000000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,22209.000000,21059.000000,19994.000000,2101.000000,2.007000e+03,1829.000000
75%,0.000000,240000.000000,2.000000,2.000000,2.000000,42.000000,0.000000,0.000000,0.000000,67270.750000,64088.000000,59942.750000,5007.000000,5.000000e+03,4500.000000
max,1.000000,800000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,746814.000000,743970.000000,693131.000000,873552.000000,1.227082e+06,889043.000000


В качестве альтернативных моделей рассмотрим SVM с различными вариантами ядер и логистическую регрессию. Причём предварительно преобразуем пространство исходных показателей с помощью метода главных компонент.

# Стандартизация и переход к главным компонентам

In [9]:

# стандартизация
sc = StandardScaler()
X_train_std = sc.fit_transform(DF.iloc[:, 1:].values)
# оцениваем объяснённую главными компонентами дисперсию
pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)
# считаем доли объяснённой дисперсии
frac_var_expl = pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',
 np.around(frac_var_expl, 3),
 '\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))


Доли объяснённой дисперсии по компонентам в PLS:
 [0.244 0.186 0.109 0.091 0.073 0.065 0.054 0.047 0.04  0.038 0.031 0.015
 0.005 0.002] 
Общая сумма долей: 1.0


Таким образом, первые две главные компоненты объясняют 40% разброса 14 объясняющих переменных. Теперь объединим функции-преобразователи и оценщики в конвейер с помощью Pipeline и оценим точность логистической регрессии с помощью перекрёстной проверки.

# Модель логистической регрессии с перекрёстной проверкой

In [10]:
X_train = DF.iloc[:, 1:]
y_train = DF.iloc[:, 0]
# объединяем в конвейер шкалирование, ГК с 2 компонентами и логит
pipe_lr = make_pipeline(StandardScaler(), PCA(n_components=4), LogisticRegression(random_state=my_seed, solver='lbfgs'))
# будем сохранять точность моделей в один массив:
score = list()
score_models = list()
# считаем точность с перекрёстной проверкой, показатель Acc
cv = cross_val_score(estimator=pipe_lr, X=X_train, y=y_train, cv=5, scoring='accuracy')
# записываем точность
score.append(np.around(np.mean(cv), 3))
score_models.append('sc_pca_logit')
print('Acc с перекрёстной проверкой',
 '\nдля модели', score_models[0], ':', score[0])



Acc с перекрёстной проверкой 
для модели sc_pca_logit : 0.798


# SVM с перекрёстной проверкой
Построим несколько вариантов модели SVM с различными ядерными функциями Так как объем данных очень большой не предоставляется возможным запустить сеточный поиск, поэтому в работе было выполнена только logit регрессия

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

pipe_svc = make_pipeline(StandardScaler(), SVC(random_state=my_seed))

# настроим параметры SVM с помощью сеточного поиска
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [{'svc__C': param_range, 'svc__kernel': ['linear']},
              {'svc__C': param_range, 'svc__gamma': param_range, 'svc__kernel': ['rbf']},
              {'svc__C': param_range, 'svc__gamma': param_range, 'svc__degree': [2, 3], 'svc__kernel': ['poly']}]

# разбиения для перекрёстной проверки
kfold = KFold(n_splits=5, random_state=my_seed, shuffle=True)

gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid,
                  scoring='accuracy', refit=True, cv=kfold, n_jobs=-1)

# таймер
tic = time.perf_counter()
# запускаем сеточный поиск
gs = gs.fit(X_train, y_train)
# таймер
toc = time.perf_counter()
print("Сеточный поиск занял {:.2f} секунд".format(toc - tic))

In [ ]:
# точность лучшей модели
np.around(gs.best_score_, 3)

In [ ]:
# параметры лучшей модели
# * ядерная функция
gs.best_estimator_.get_params()['svc__kernel']

In [ ]:
# * параметр регуляризации
gs.best_estimator_.get_params()['svc__C']

In [ ]:
# * коэффициент ядерной функции (для ядер 'rbf', 'poly' и 'sigmoid')
gs.best_estimator_.get_params()['svc__gamma']

In [ ]:
# * степень полинома (для ядра 'poly')
gs.best_estimator_.get_params()['svc__degree']

In [ ]:
# сводка по точности моделей
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

In [ ]:
# прогноз с помощью лучшей модели ансамбля с SVC
y_hat = gs.best_estimator_.predict(X=DF_predict.loc[:, 2:])
# точность
# характеристики точности
print(classification_report(DF_predict.loc[:, 1], y_hat))